# Benchmarking TSP on different solvers

https://arxiv.org/pdf/2105.02208

### Initial setups

In [1]:
import sys
sys.path.append('..')

In [2]:
from utils import get_distance_matrices
from travelingsalesperson import TSP
from Utils.solvers import GetGurobiClient, GetFixstarClient, GetDWaveClient
from plot import RunData
from amplify import solve

Nodes = [4, 5, 6, 7, 8, 10, 12, 15, 20, 69]

problems = get_distance_matrices(Nodes)

# print(problems)


### Run all

In [3]:
# for window with conda env
clientG = GetGurobiClient(r"D:\miniconda3\envs\cwq\gurobi110.dll")
clientFS = GetFixstarClient()
clientDWave = GetDWaveClient()
clientDWave.parameters.num_reads = 3_000

benchmark_result = RunData()

def run(node: int, penalty: int = 1_000_000):
    problem = problems['dense']['low'][str(node)]
    
    tsp = TSP.from_distance_matrix(problem)
    
    qp, formulating_time = tsp.qubo(format="amplify", penalty=penalty)
    
    model = qp["model"]
    
    try:
        try:
            # gurobi
            resultG = solve(model, clientG)
            durG, costG, _ = tsp.interpret(resultG, solver="Gurobi", verbose=True)
        except Exception as e:
            print(e)
            durG, costG = None, None
            print("Gurobi failed")
        
        try:   
            # fixstar
            resultF = solve(model, clientFS)
            durF, costF, _ = tsp.interpret(resultF, solver="Fixstar", verbose=True)
        except Exception as e:
            print(e)
            durF, costF = None, None
            print("Fixstar failed")

        try: 
            resultD = solve(model, clientDWave)
            durD, costD, _ = tsp.interpret(resultD, solver="D-Wave", verbose=True)
        except Exception as e:
            print(e)
            durD, costD = None, None
            print("Dwave failed")

        
        benchmark_result.add(node, problem, penalty, formulating_time, costG, durG, costF, durF, costD, durD)
    except KeyboardInterrupt as e:
        print("quit")
        
    
        

In [4]:
for node in Nodes:
    print(f"====================== Node: {node} ======================")
    run(node)
    
benchmark_result.save()

====================== Node: 4 ======================
Total time taken in :  qubo 0.0
Cost: 21.0
Optimal tour: [0. 1. 2. 3.]
Execution time: 0.013999
Cost: 21.0
Optimal tour: [2. 1. 0. 3.]
Execution time: 0.957786
Cost: 21.0
Optimal tour: [1. 0. 3. 2.]
Execution time: 0.608261
====================== Node: 5 ======================
Total time taken in :  qubo 0.0
Cost: 16.0
Optimal tour: [2. 3. 4. 0. 1.]
Execution time: 0.016
Cost: 16.0
Optimal tour: [2. 1. 0. 4. 3.]
Execution time: 0.99791
Cost: 16.0
Optimal tour: [4. 0. 1. 3. 2.]
Execution time: 0.628779
====================== Node: 6 ======================
Total time taken in :  qubo 0.0
Cost: 16.0
Optimal tour: [1. 2. 4. 0. 5. 3.]
Execution time: 0.021999
Cost: 16.0
Optimal tour: [5. 3. 1. 2. 4. 0.]
Execution time: 1.000638
Cost: 16.0
Optimal tour: [4. 0. 5. 3. 1. 2.]
Execution time: 0.663341
====================== Node: 7 ======================
Total time taken in :  qubo 0.0010101795196533203
Cost: 26.0
Optimal tour: [0. 1. 3. 2. 5

In [5]:
benchmark_result.results

[{'nodes': 4,
  'max_edge_weight': 8,
  'avg_edge_weight': 6.0,
  'distance_matrix': [[0, 5, 8, 6], [5, 0, 5, 7], [8, 5, 0, 5], [6, 7, 5, 0]],
  'qp_weight': 1000000,
  'time_model_formulation': 0.0,
  'gurobi_objective': 21.0,
  'gurobi_execution_time': 0.013999,
  'fixstars_objective': 21.0,
  'fixstars_execution_time': 0.957786,
  'dwave_objective': 21.0,
  'dwave_execution_time': 0.608261},
 {'nodes': 5,
  'max_edge_weight': 8,
  'avg_edge_weight': 4.3,
  'distance_matrix': [[0, 3, 6, 7, 4],
   [3, 0, 1, 4, 8],
   [6, 1, 0, 3, 2],
   [7, 4, 3, 0, 5],
   [4, 8, 2, 5, 0]],
  'qp_weight': 1000000,
  'time_model_formulation': 0.0,
  'gurobi_objective': 16.0,
  'gurobi_execution_time': 0.016,
  'fixstars_objective': 16.0,
  'fixstars_execution_time': 0.99791,
  'dwave_objective': 16.0,
  'dwave_execution_time': 0.628779},
 {'nodes': 6,
  'max_edge_weight': 9,
  'avg_edge_weight': 4.866666666666666,
  'distance_matrix': [[0, 6, 9, 7, 4, 2],
   [6, 0, 1, 3, 5, 7],
   [9, 1, 0, 7, 4, 6],
 